In [9]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient

In [16]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '440'
start = '2016-01-01T00:00:00Z'
end = '2021-06-013T00:00:00Z'
mongo_org_collection = db[str(organization_id)]
escalation = ["roadie_escalations", "ubiquity_escalation"]
esc = "forethought_escalations"
nma = "no macro applied"
invalid = {'locked', '10_user_education', '20_account_creation/maintenance/cancellation', '90_internal_task'}

In [17]:
def get_label(i):
    return esc if "[Fraud-Request]" not in i['subject'] and all(x not in i['tags'] for x in invalid) else nma

In [18]:
def get_dataset():
    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': {'$in':['web','email', 'api']},
        'original_item.created_at': {'$gte': start, '$lte': end},
        "original_item.tags": {"$in": escalation}
    })
    
    file_path = os.path.expanduser('~/Downloads/rdie_esc_new.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            i = ticket["original_item"]
            variable_labels = [x for x in escalation if x in i['tags']]

            j = {
                "ticket_id": i['id'],
                "text_a": i["subject"],
                "text_b": i["description"],
                "created_at": i["created_at"],
                # "label": variable_labels[0] if len(variable_labels) > 0 else None,
                "old_label": sorted(variable_labels),
                "label": get_label(i),
                "tags": i['tags']
                }
                
            json.dump(j, f)
            _ = f.write('\n')
            count += 1
            if count % 100 == 0:
                print(count)

    print("total: ", count)


get_dataset()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
total:  1858
